In [20]:
import sqlite3
import pandas as pd

db = sqlite3.connect('AMS.db')
query = 'select * from Order_Table'
df = pd.read_sql(query, con = db)
df = df[df['TradeID']==1]
df.groupby(['Date','OrderID','TradeID'])[['SettleAmt']].sum()

SettleAmt
Date       OrderID TradeID           
2019-12-02 1       1            185.8
           2       1             35.8
2019-12-03 1       1            375.8
2019-12-05 1       1           -287.2
           2       1            362.8
           3       1            122.8
           4       1             12.8

In [21]:
#df.groupby(['Date'])[['SettleAmt']].sum()